In [11]:
import numpy as np
import sklearn
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import nltk # just for tokenization
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string

In [12]:
log = ""

In [13]:
def precision(y_true, y_pred, num_classes):
    # Initialize arrays to store true positives, false positives, and precision
    TP = np.zeros(num_classes)
    FP = np.zeros(num_classes)
    precision_scores = np.zeros(num_classes)

    # Calculate true positives and false positives for each class
    for i in range(num_classes):
        TP[i] = np.sum((y_true == i) & (y_pred == i))
        FP[i] = np.sum((y_true != i) & (y_pred == i))

    # Compute precision for each class
    for i in range(num_classes):
        if TP[i] + FP[i] > 0:
            precision_scores[i] = TP[i] / (TP[i] + FP[i])

    return np.mean(precision_scores)

In [15]:
def hamming_loss(y_true, y_pred):
    # Calculate number of mismatches
    num_mismatches = np.sum(y_true != y_pred)

    # Compute Hamming Loss
    hamming_loss = num_mismatches / (y_true.shape[0] * y_true.shape[1])

    return hamming_loss

In [49]:
def top2_accuracy(predicted_probs, true_labels):

    sorted_indices = np.argsort(predicted_probs, axis=1)[:, ::-1]

    # Check if true labels are in top-3 predicted labels
    top3_correct = np.any(true_labels[np.arange(len(true_labels))[:, None], sorted_indices[:, :2]], axis=1)
    # Calculate top-3 accuracy
    top3_accuracy = np.mean(top3_correct)
    
    return top3_accuracy

In [50]:
class NeuralNetwork:
    
    def __init__(self, raw_data, embeddings):
        
        self.raw_data = raw_data
        self.random_state = 42
        
        self.X_train, self.X_test, self.y_train, self.y_test = None, None, None, None
        self.labels = ['Joy', 'Trust', 'Fear', 'Surprise','Sadness', 'Disgust', 'Anger', 'Anticipation']
        self.emotions_onehot = np.array(raw_data.loc[:, self.labels])
        self.__pre_process(embeddings)
        
        self.n_classes = self.y_train.shape[1]
        self.n_input_features = self.X_train.shape[1]
        self.n_hidden_neurons = 3
        
        
        #weights from input layer to hidden layer1
        np.random.seed(self.random_state)
        self.W01 = np.random.randn(self.n_input_features, self.n_hidden_neurons)
        self.W12 = np.random.randn(self.n_hidden_neurons, self.n_classes)
        
        self.b01 = np.zeros((1, self.n_hidden_neurons))
        self.b12 = np.zeros((1, self.n_classes))
        
    def __activation(self, activation_function, X):
        if activation_function == "sigmoid":
            return 1/(1+np.exp(-X))
        elif activation_function == "relu":
            return (X > 0) * X
        elif activation_function == "tanh":
            return (np.exp(X) + np.exp(-X))/(np.exp(X) - np.exp(-X))
        
    def __activation_derivative(self, activation_function, X):
        if activation_function == "sigmoid":
            return self.__activation(activation_function, X) * (1 - self.__activation(activation_function, X))
        elif activation_function == "relu":
            return X > 0
        elif activation_function == "tanh":
            return 1 - self.__activation(activation_function, X)**2
    
    def __error(self, preds, ground, error="mean"):
        return -np.mean(ground * np.log(preds) + (1 - ground) * np.log(1 - preds))
        # return 0.5 * preds.shape[1] * ((ground - preds)**2).sum()
        
    
    #Note: output activation will always be sigmoid
    def train(self, epochs=100, lr = 1e-1, hidden_layer_activation = "relu", batch_size = 16, thresold = 0.6):
        global log
        log += f"{self.n_hidden_neurons}, {batch_size},"
        log += f"{epochs}, {lr}, {hidden_layer_activation},"
        # forward the data, and then calculate the training accuracy
        self.hidden_layer_activation = hidden_layer_activation
        print(f"number of batches {self.X_train.shape[0]//batch_size}")
        error = 0
        for epoch in range(epochs):
            #batch gd
            batches = (self.X_train.shape[0] % batch_size)
            exact_batches = True if batches == 0 else False
            n_batches = (self.X_train.shape[0]//batch_size) if exact_batches else (self.X_train.shape[0]//batch_size + 1)
            for batch in range(n_batches):
                b = batch*batch_size
                b_1 = self.X_train.shape[0] if (not exact_batches) and (batch == n_batches-1) else (batch+1)*batch_size
                self.X_batch = self.X_train[b:b_1]
                self.Y_batch = self.y_train[b:b_1]
                self.Z01 = self.X_batch.dot(self.W01) + self.b01
                self.A01 = self.__activation(hidden_layer_activation, self.Z01)
                self.Z02 = self.A01.dot(self.W12) + self.b12
                self.A02 = self.__activation("sigmoid", self.Z02)
                
                error = self.__error(self.A02, self.Y_batch)

                self.backward()

                self.W12 -= lr * self.A01.T.dot(self.d_error_W12)
                self.b12 -= lr * np.sum(self.d_error_W12, axis=0, keepdims = True)
                self.W01 -= lr * self.X_batch.T.dot(self.d_error_W01)

            # print(f"Error {error}")
            if epoch % 10 == 0:
                self.test(epoch, self.X_train, self.y_train)
                print(f"Error {error}")
                # train_accuracy = self.accuracy(self.A02, self.Y_batch)
                # print(self.Y_batch, b, b_1)

                
        # log += f"{train_accuracy},"
    def backward(self):
        self.d_error_A02 = (self.A02 - self.Y_batch)/len(self.Y_batch)
        self.d_error_W12 = (self.d_error_A02) * self.__activation_derivative("sigmoid", self.Z02)
        
        self.d_error_W01 = (
            (self.d_error_W12).dot(self.W12.T) * self.__activation_derivative(self.hidden_layer_activation, self.Z01))
    
    
    def __pre_process(self, embeddings, train_test_ratio = 0.3):
        # self.raw_data = self.raw_data.drop("Id",axis=1)
        # species_np = np.array(self.raw_data["Species"])
        # onehotencoder  = OneHotEncoder(sparse_output = False)
        # target_onehot = onehotencoder.fit_transform(species_np.reshape(-1,1))
        # self.raw_data=self.raw_data.drop("Species", axis=1)
        
        X = embeddings 
        y = self.emotions_onehot
        
        self.X_train, self.X_test, self.y_train, self.y_test = X, X, y, y
  

    def accuracy(self, y_pred, y_ground):
        y_train_predicted_classes = np.argmax(y_pred, axis = 1)
        y_train_ground_classes = np.argmax(y_ground, axis=1)
        accuracy = ((y_train_predicted_classes == y_train_ground_classes).sum())/len(y_train_predicted_classes)
        return accuracy
               
    def test(self, epoch, X, y):
        global log
        Z01 = X.dot(self.W01) + self.b01
        A01 = self.__activation(self.hidden_layer_activation, Z01)
        Z02 = A01.dot(self.W12) + self.b12
        A02 = self.__activation("sigmoid", Z02)
        predictions = A02.round()
        precision_metric = precision(y, predictions, y.shape[1])
        hamming_loss_metric = hamming_loss(y, predictions)
        top3_metric = top2_accuracy(A02, y)
        print(f"Epoch {epoch}, Precision {precision_metric}, top3metric {top3_metric}, Hamming loss {hamming_loss_metric}")

    def predict(self, X):
        Z01 = X.dot(self.W01) + self.b01
        A01 = self.__activation(self.hidden_layer_activation, Z01)
        Z02 = A01.dot(self.W12) + self.b12
        A02 = self.__activation("sigmoid", Z02)
        return A02

        
    def print_shapes(self):
        print(f"Xtrain shape {self.X_train.shape}")
        print(f"ytrain shape {self.y_train.shape}")
        print(f"Xtest shape {self.X_test.shape}")
        print(f"ytest shape {self.y_test.shape}")
    
        

# Preprocessing

In [51]:
raw_data = pd.read_csv("../../data/EdmondsDance.csv")

In [52]:
#remove unwanted columns
raw_data.pop("Unnamed: 0")
raw_data.pop("Unnamed: 11")

0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
       ..
519   NaN
520   NaN
521   NaN
522   NaN
523   NaN
Name: Unnamed: 11, Length: 524, dtype: float64

In [53]:
raw_data.head()

,Song,Artists,Lyrics,Joy,Trust,Fear,Surprise,Sadness,Disgust,Anger,Anticipation
0,Apollo,"Hardwell, Amba Shepherd",Just one day in the life<br>So I can understan...,1,1,0,1,0,0,0,0
1,Lullaby,"R3HAB, Mike Williams","Hypnotized, this love out of me<br>Without you...",0,0,1,0,1,0,0,0
2,Melody (Tip Of My Tongue),Mike Williams,I stand a little too close<br>You stare a litt...,1,1,0,0,0,0,0,1
3,Take Me Home,"Cash Cash, Bebe Rexha",I'm falling to pieces<br>But I need this<br>Ye...,0,0,0,1,1,1,0,0
4,City of Dreams,"Dirty South, Alesso","Everything seems like a city of dreams,<br>I n...",0,0,0,1,1,0,0,0


In [24]:
raw_data.describe()

,Joy,Trust,Fear,Surprise,Sadness,Disgust,Anger,Anticipation
count,524.000000,524.000000,524.000000,524.000000,524.000000,524.000000,524.000000,524.000000
mean,0.438931,0.561069,0.196565,0.129771,0.353053,0.219466,0.137405,0.475191
std,0.496731,0.496731,0.397780,0.336372,0.478376,0.414280,0.344603,0.499861
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [25]:
def load_embedding_model():
    """ Load GloVe Vectors
        Return:
            wv_from_bin: All 400000 embeddings, each lengh 200
    """
    import gensim.downloader as api
    wv_from_bin = api.load("word2vec-google-news-300")
    print("Loaded vocab size %i" % len(list(wv_from_bin.index_to_key)))
    return wv_from_bin
wv_from_bin = load_embedding_model()
# wv_from_bin = load_embedding_model()

Loaded vocab size 3000000


In [31]:
def tokenize(lyric: str) -> list[str]:
    # lowercase the text, remove stop words, punctuation and keep only the words
    lyric.replace("<br>", "\n")
    tokens = nltk.tokenize.word_tokenize(lyric.lower())
    stop_words = stopwords.words("english") + list(string.punctuation)
    lemmatizer = WordNetLemmatizer()
    alpha_tokens = [lemmatizer.lemmatize(token) for token in tokens if token.isalpha() and token not in stop_words]

    return alpha_tokens

In [32]:
def vectorise(lyrics: str) -> np.ndarray:
    tokens = tokenize(lyrics)
    lyric_vector = np.zeros(300)
    for token in tokens:
        try:
            lyric_vector += wv_from_bin.get_vector(token.lower())
        except:
            continue
    return lyric_vector / np.linalg.norm(lyric_vector)

In [33]:
raw_data["Lyrics"]

0      Just one day in the life<br>So I can understan...
1      Hypnotized, this love out of me<br>Without you...
2      I stand a little too close<br>You stare a litt...
3      I'm falling to pieces<br>But I need this<br>Ye...
4      Everything seems like a city of dreams,<br>I n...
                             ...                        
519    ashes to ashes<br>we're falling down<br>so we ...
520    I want to hold you<br>I want to hold you<br>I ...
521    There's not enough room in here<br>For room fo...
522    I see you everywhere<br>I never moved on<br>Wi...
523    These stars are, really fireflies that lost th...
Name: Lyrics, Length: 524, dtype: object

In [34]:
# go through each lyrics, tokenize it, vectorize each word, then combine all of them into single average vector and store it in the list
lyrics = raw_data["Lyrics"]
lyrics_embeddings = []
unsupported_tokens = set()
label_embedding_map = {} # dict{str: np.array([])}
for lyric in tqdm(lyrics):
    lyric_vector = np.zeros(300)
    for token in tokenize(lyric):
        try:
            lyric_vector += wv_from_bin.get_vector(token.lower())
        except KeyError as e:
            # if the word is not present in the glove then key error is raised, so handle the exception and move on
            unsupported_tokens.add(token)
            continue
    lyrics_embeddings.append(lyric_vector)


lyrics_embeddings = np.stack(lyrics_embeddings)
scaled_lyrics_embeddings = lyrics_embeddings / np.linalg.norm(lyrics_embeddings, axis=1, keepdims=True)

100%|██████████| 524/524 [00:01<00:00, 358.47it/s]


# Train

In [60]:
nn = NeuralNetwork(raw_data, scaled_lyrics_embeddings)

In [61]:
nn.train(epochs=1000, lr=1e-1, hidden_layer_activation="relu", batch_size=8)

number of batches 65
Epoch 0, Precision 0.15140561935834013, top3metric 0.7862595419847328, Hamming loss 0.3485209923664122
Error 0.5754508802629186
Epoch 10, Precision 0.16007040319893995, top3metric 0.7385496183206107, Hamming loss 0.29842557251908397
Error 0.5064143343727087
Epoch 20, Precision 0.15996496781884603, top3metric 0.7423664122137404, Hamming loss 0.2989026717557252
Error 0.5087259206703109
Epoch 30, Precision 0.16027862652480718, top3metric 0.7461832061068703, Hamming loss 0.29842557251908397
Error 0.5114422207002614
Epoch 40, Precision 0.16026708075153362, top3metric 0.75, Hamming loss 0.29842557251908397
Error 0.5134101403586184
Epoch 50, Precision 0.1611741519350215, top3metric 0.7538167938931297, Hamming loss 0.2967557251908397
Error 0.5149822771973102
Epoch 60, Precision 0.16054835459714997, top3metric 0.7595419847328244, Hamming loss 0.29770992366412213
Error 0.516273799999355
Epoch 70, Precision 0.16185717987395776, top3metric 0.7652671755725191, Hamming loss 0.29

In [62]:
song = """
I heard that you're settled down
That you found a girl and you're married now
I heard that your dreams came true
Guess she gave you things, I didn't give to you
Old friend, why are you so shy?
Ain't like you to hold back or hide from the light
I hate to turn up out of the blue, uninvited
But I couldn't stay away, I couldn't fight it
I had hoped you'd see my face
And that you'd be reminded that for me, it isn't over
Never mind, I'll find someone like you
I wish nothing but the best for you, too
"Don't forget me, " I beg
I remember you said
"Sometimes it lasts in love, but sometimes it hurts instead"
"Sometimes it lasts in love, but sometimes it hurts instead"
You know how the time flies
Only yesterday was the time of our lives
We were born and raised in a summer haze
Bound by the surprise of our glory days
I hate to turn up out of the blue, uninvited
But I couldn't stay away, I couldn't fight it
I had hoped you'd see my face
And that you'd be reminded that for me, it isn't over
Never mind, I'll find someone like you
I wish nothing but the best for you, too
"Don't forget me, " I begged
I remember you said
"Sometimes it lasts in love, but sometimes it hurts instead"
Nothing compares, no worries or cares
Regrets and mistakes, they're memories made
Who would have known how bittersweet this would taste?
Never mind, I'll find someone like you
I wish nothing but the best for you
"Don't forget me, " I beg
I remember you said
"Sometimes it lasts in love, but sometimes it hurts instead"
Never mind, I'll find someone like you
I wish nothing but the best for you, too
"Don't forget me, " I begged
I remember you said
"Sometimes it lasts in love, but sometimes it hurts instead"
"Sometimes it lasts in love, but sometimes it hurts instead"
"""

In [63]:
def predict(lyrics: str) -> str:
    song_vector = vectorise(lyrics)[None,:]
    return nn.predict(song_vector)

In [64]:
probs = predict(song)

In [65]:
probs

array([[0.02755619, 0.14568433, 0.5158429 , 0.24160766, 0.89812024,
        0.66938076, 0.30923268, 0.22966507]])

In [66]:
np.argsort(probs[0])[::-1]

array([4, 5, 2, 6, 3, 7, 1, 0], dtype=int64)

In [74]:
nn.labels

['Joy',
 'Trust',
 'Fear',
 'Surprise',
 'Sadness',
 'Disgust',
 'Anger',
 'Anticipation']

In [68]:
nn.predict

<bound method NeuralNetwork.predict of <__main__.NeuralNetwork object at 0x000002B920210190>>

In [69]:
np.array(nn.labels)[np.argsort(probs[0])[::-1]]

array(['Sadness', 'Disgust', 'Fear', 'Anger', 'Surprise', 'Anticipation',
       'Trust', 'Joy'], dtype='<U12')

In [71]:
import pickle

with open("../embeddings/nn.pickle", "wb") as f:
    pickle.dump(nn, f)

In [72]:
import pickle


with open("../embeddings/nn.pickle", "rb") as f:
    a = pickle.load(f)

In [73]:
a.labels

['Joy',
 'Trust',
 'Fear',
 'Surprise',
 'Sadness',
 'Disgust',
 'Anger',
 'Anticipation']